In [3]:
import pandas as pd
import numpy as np
from datetime import datetime

In [4]:
raw_data = pd.read_json("ratebeer_korea.json")
raw_data = raw_data[~raw_data.reviewScore.isna()]
raw_data.head(3)

,reviewScore,reviewTime,reviewText,appearance,aroma,palate,taste,overall,profileName,beerName,beerID,brewerID,ABV,style,imageUrl
0,1.3,2021-02-27 12:53:53.373,Small can from Valencia. Thanks Mario for this...,2.0,1.0,1.0,3.0,6.0,Grzesiek79,Eichbaum Apostel Bräu,75661,1103,5.0,Pilsener / Pils / Pilsner,https://res.cloudinary.com/ratebeer/image/uplo...
1,2.7,2021-01-08 18:06:32.240,"Ar: slod, niska intensywnosc, sladowy slod. Wy...",4.0,5.0,3.0,5.0,10.0,GriZZZly,Eichbaum Apostel Bräu,75661,1103,5.0,Pilsener / Pils / Pilsner,https://res.cloudinary.com/ratebeer/image/uplo...
2,3.3,2020-12-17 14:20:21.580,More full bodied than most no doubt. Not reall...,NaN,NaN,NaN,NaN,NaN,bdleedahl,Eichbaum Apostel Bräu,75661,1103,5.0,Pilsener / Pils / Pilsner,https://res.cloudinary.com/ratebeer/image/uplo...


In [6]:
reviewer.user_id

0            0
1            1
2            2
3            3
4            4
         ...  
17160    17160
17161    17161
17162    17162
17163    17163
17164    17164
Name: user_id, Length: 17165, dtype: int64

In [5]:
n_user = raw_data["profileName"].nunique()

#### DB-name : reviewer ####
reviewer = pd.DataFrame()
reviewer["user_id"] = range(n_user)
# reviewer["email"] = raw_data["profileName"].unique() + "@ratebeer.com"
reviewer["password"] = [str(num).zfill(8) for num in np.random.randint(low = 0, high = 99999999, size = n_user)]
reviewer["profile_name"] = raw_data["profileName"].unique()
reviewer["gender"] = "-"
reviewer["birth"] = [datetime.strptime("1000-01-01 00:00:00.001", "%Y-%m-%d %H:%M:%S.%f")]*n_user

print("n_user :", n_user)
reviewer.head()

n_user : 17165


,user_id,password,profile_name,gender,birth
0,0,28888657,Grzesiek79,-,1000-01-01 00:00:00.001000
1,1,80569693,GriZZZly,-,1000-01-01 00:00:00.001000
2,2,75415240,bdleedahl,-,1000-01-01 00:00:00.001000
3,3,60066083,DoctorHU,-,1000-01-01 00:00:00.001000
4,4,12433786,McTapps,-,1000-01-01 00:00:00.001000


In [139]:
#### DB-name : beer ####
beer = raw_data.groupby("beerName")[["beerID","beerName", "brewerID", "ABV", "style", "imageUrl"]].head(1).copy()
beer.columns = ["beer_id", "beer_name", "brewer_id", "abv", "style", "image_url"]
beer.index = range(len(beer))

print("n_beer :",  beer.shape[0])
beer.head()

n_beer : 81


,beer_id,beer_name,brewer_id,abv,style,image_url
0,75661,Eichbaum Apostel Bräu,1103,5.0,Pilsener / Pils / Pilsner,https://res.cloudinary.com/ratebeer/image/uplo...
1,614833,Apple Fox,2937,4.5,Apple Cider,https://res.cloudinary.com/ratebeer/image/uplo...
2,169964,Asahi Super Dry Black,50,5.5,Dark Lager - International / Premium,https://res.cloudinary.com/ratebeer/image/uplo...
3,251,Asahi Super Dry,50,5.0,Pale Lager - International / Premium,https://res.cloudinary.com/ratebeer/image/uplo...
4,10726,Bali Hai,1974,4.9,Pale Lager - American,https://res.cloudinary.com/ratebeer/image/uplo...


In [140]:
#### DB-name : review ####
profileName_with_userid = reviewer[["profile_name", "user_id"]].rename(columns={"profile_name":"profileName"})
raw_data_with_userid = raw_data.merge(profileName_with_userid, on = "profileName")
review = raw_data_with_userid[["user_id", "beerID", "reviewScore", "reviewText", "reviewTime", "appearance", "aroma", "palate", "taste", "overall"]]
review.columns = ["user_id", "beer_id", "reviewscore", "reviewtext", "reviewtime", "appearance", "aroma", "palate", "taste", "overall"]

review.head()

,user_id,beer_id,reviewscore,reviewtext,reviewtime,appearance,aroma,palate,taste,overall
0,0,75661,1.3,Small can from Valencia. Thanks Mario for this...,2021-02-27 12:53:53.373,2.0,1.0,1.0,3.0,6.0
1,0,169964,2.8,"Dry, czyli wytrawne. Sadzilem ze bedzie jednak...",2016-08-20 08:34:12.010,3.0,4.0,2.0,5.0,14.0
2,0,251,2.2,2l Big Can. Karakuchi write on this can. Brewe...,2012-03-24 13:01:56.070,2.0,3.0,2.0,5.0,10.0
3,0,5588,3.9,"barnyard, grass, cereal, rotten eggs, orange, ...",2012-03-23 09:54:23.480,3.0,7.0,4.0,8.0,17.0
4,0,5571,2.1,Bavaria Pilsener - 500ml green bottle (green l...,2013-08-18 08:38:06.850,3.0,3.0,1.0,5.0,9.0


In [152]:
# review.to_json("db_review.json")

reviewer.to_csv("db_reviewer.csv", index = False)
beer.to_csv("db_beer.csv", index = False)

review_new = review.copy()

review_new["reviewtext"] = [texts.replace("\"", " ") for texts in review["reviewtext"]]
review_new.to_csv("db_review.csv",index = False, sep="|")

# review_new.to_csv("db_review.csv",index = False, sep=",")

In [153]:
count = 0
for i in review_new["reviewtext"]:
    if "\"" in i: 
        count += 1



In [154]:
count

0

In [119]:
# beer.nunique()

In [117]:
# # beer["style"]
# beer.groupby("style")["beer_id"].nunique()#.sum()

In [118]:
# beer_style = beer.copy()
# beer_style["large_style"] = [ styles.split(" - ")[0] for styles in beer["style"] ]
# beer_style["large_style"] = [ styles.split(" / ")[0] for styles in beer_style["large_style"] ]
# beer_style.groupby("large_style")["beer_id"].nunique()